# 隐私政策分析系统结果对比框架

这个笔记本用于对比：
1. **我们的RAG系统** vs **Ground Truth**
2. **ChatGPT** vs **Ground Truth**

## 表格格式要求
- 按问题分组显示：先显示所有应用的Q1，再显示所有应用的Q3，然后Q4，最后Q5
- 不显示Q2和Q6
- 数据来源：`new version/outputs` 文件夹中的RAG数据
- 输出位置：`new version/outputs` 文件夹

In [1]:
# 📊 RAG系统与Ground Truth对比表格生成
import pandas as pd
import json
import os
import json
from pathlib import Path

project_root = Path(__file__).resolve().parent if '__file__' in globals() else Path.cwd()
def create_comparison_tables():
    """
    创建RAG系统和ChatGPT与Ground Truth的对比表格
    按问题分组显示：Q1→Q3→Q4→Q5
    """
    print("🚀 开始生成对比表格...")
    
    # 1. 定义文件路径

    


    gt_file = str(project_root /"groundtruth.json")

    new_version_outputs = project_root/"new version"/"outputs"
    chatgpt_outputs = project_root/"new version"/"gptoutputs"
    output_dir=project_root/"final outputs"
    
    # 2. 加载 Ground Truth 数据
    try:
        with open(gt_file, 'r', encoding='utf-8') as f:
            gt_data = json.load(f)
        print(f"✅ 成功加载 Ground Truth: {len(gt_data)} 条记录")
    except Exception as e:
        print(f"❌ 无法加载 Ground Truth: {e}")
        return None, None
    
    # 3. 获取所有RAG JSON文件
    rag_files = []
    try:
        for file in os.listdir(new_version_outputs):
            if file.endswith('.json') and not file.startswith('rag_') and not file.startswith('question_') and not file.startswith('app_') and not file.startswith('error_'):
                rag_files.append(file)
        print(f"✅ 找到 {len(rag_files)} 个RAG数据文件")
        print(f"📁 文件列表: {rag_files}")
    except Exception as e:
        print(f"❌ 无法读取RAG文件目录: {e}")
        return None, None
    
    # 4. 收集所有应用的数据
    all_apps_data = []
    
    for rag_filename in rag_files:
        rag_filepath = os.path.join(new_version_outputs, rag_filename)
        
        try:
            # 加载RAG数据
            with open(rag_filepath, 'r', encoding='utf-8') as f:
                rag_data = json.load(f)
            
            # 提取应用信息
            app_id = rag_data[0]['meta']['id']
            app_url = rag_data[0]['meta']['url']
            app_name = app_url.replace('http://', '').replace('https://', '').split('/')[0]
            
            print(f"📱 处理应用: {app_name} (ID: {app_id})")
            
            # 查找对应的Ground Truth
            gt_record = None
            for gt in gt_data:
                if 'id' in gt and gt['id'] == app_id:
                    gt_record = gt
                    break
            
            if not gt_record:
                print(f"   ⚠️ 未找到应用ID {app_id} 的Ground Truth，跳过")
                continue
            
            # 提取RAG答案
            rag_answers = {}
            for item in rag_data:
                question_text = item['reply']['question']
                
                # 根据问题文本判断问题编号
                if question_text.startswith('1.'):
                    qid = 'q1'
                elif question_text.startswith('3.'):
                    qid = 'q3'
                elif question_text.startswith('4.'):
                    qid = 'q4'
                elif question_text.startswith('5.'):
                    qid = 'q5'
                else:
                    continue
                
                # 获取simple_answer
                if 'simple_answer' in item['reply']['answer'] and item['reply']['answer']['simple_answer']:
                    rag_answers[qid] = item['reply']['answer']['simple_answer']
                else:
                    rag_answers[qid] = 'Yes'  # 默认值
            
            # 标准化答案函数
            def normalize_answer(answer):
                if str(answer).lower() == 'y':
                    return 'Yes'
                elif str(answer).lower() == 'n':
                    return 'No'
                return str(answer)
            
            # 保存应用数据
            app_data = {
                'app_name': app_name,
                'app_id': app_id,
                'answers': {}
            }
            
            questions = ['q1', 'q3', 'q4', 'q5']
            for qid in questions:
                # Ground Truth答案
                gt_answer_raw = gt_record.get(qid, 'n')
                gt_answer = normalize_answer(gt_answer_raw)
                
                # RAG答案
                rag_answer = rag_answers.get(qid, 'No')
                
                app_data['answers'][qid] = {
                    'gt_answer': gt_answer,
                    'rag_answer': rag_answer
                }
            
            all_apps_data.append(app_data)
            print(f"   ✅ 处理完成")
        
        except Exception as e:
            print(f"   ❌ 处理文件 {rag_filename} 时出错: {e}")
            continue
    
    # 5. 创建RAG系统对比表格（按问题分组）
    rag_comparisons = []
    questions_order = ['q1', 'q3', 'q4', 'q5']
    question_labels = {'q1': 'Q1', 'q3': 'Q3', 'q4': 'Q4', 'q5': 'Q5'}
    
    print(f"\n📋 创建RAG系统对比表格（按问题分组）...")
    
    for qid in questions_order:
        question_label = question_labels[qid]
        print(f"   添加所有应用的 {question_label} 问题...")
        
        for app_data in all_apps_data:
            if qid in app_data['answers']:
                answer_data = app_data['answers'][qid]
                
                rag_comparisons.append({
                    'Question': question_label,
                    'App': app_data['app_id'],
                    'Ground Truth': answer_data['gt_answer'],
                    'Our RAG System': answer_data['rag_answer']
                })
    
    # 6. 检查ChatGPT数据是否存在
    chatgpt_comparisons = []
    chatgpt_available = False
    
    print(f"\n📋 检查ChatGPT数据...")
    
    if os.path.exists(chatgpt_outputs):
        # 获取ChatGPT文件
        chatgpt_files = []
        try:
            for file in os.listdir(chatgpt_outputs):
                if file.endswith('.json'):
                    chatgpt_files.append(file)
            
            if len(chatgpt_files) > 0:
                print(f"✅ 找到 {len(chatgpt_files)} 个ChatGPT数据文件")
                chatgpt_available = True
                
                # 处理ChatGPT数据
                print(f"📝 开始处理ChatGPT数据...")
                
                # 收集所有ChatGPT应用的数据
                chatgpt_apps_data = []
                
                for chatgpt_filename in chatgpt_files:
                    chatgpt_filepath = os.path.join(chatgpt_outputs, chatgpt_filename)
                    
                    try:
                        # 加载ChatGPT数据
                        with open(chatgpt_filepath, 'r', encoding='utf-8') as f:
                            chatgpt_data = json.load(f)
                        
                        # 提取应用信息
                        app_id = chatgpt_data[0]['meta']['id']
                        print(f"📱 处理ChatGPT应用: ID {app_id}")
                        
                        # 查找对应的Ground Truth
                        gt_record = None
                        for gt in gt_data:
                            if 'id' in gt and gt['id'] == app_id:
                                gt_record = gt
                                break
                        
                        if not gt_record:
                            print(f"   ⚠️ 未找到应用ID {app_id} 的Ground Truth，跳过")
                            continue
                        
                        # 提取ChatGPT答案
                        chatgpt_answers = {}
                        for item in chatgpt_data:
                            qid_num = item['reply']['qid']
                            
                            # 将数字qid映射到我们使用的格式
                            if qid_num == '1':
                                qid = 'q1'
                            elif qid_num == '3':
                                qid = 'q3'
                            elif qid_num == '4':
                                qid = 'q4'
                            elif qid_num == '5':
                                qid = 'q5'
                            else:
                                continue
                            
                            # 获取simple_answer
                            if 'simple_answer' in item['reply']['answer'] and item['reply']['answer']['simple_answer']:
                                answer_text = item['reply']['answer']['simple_answer']
                                # 从答案中提取Yes/No
                                if 'Yes' in answer_text:
                                    chatgpt_answers[qid] = 'Yes'
                                elif 'No' in answer_text:
                                    chatgpt_answers[qid] = 'No'
                                else:
                                    chatgpt_answers[qid] = 'Yes'  # 默认值
                            else:
                                chatgpt_answers[qid] = 'Yes'  # 默认值
                        
                        # 保存ChatGPT应用数据
                        chatgpt_app_data = {
                            'app_id': app_id,
                            'answers': {}
                        }
                        
                        questions = ['q1', 'q3', 'q4', 'q5']
                        for qid in questions:
                            # Ground Truth答案
                            gt_answer_raw = gt_record.get(qid, 'n')
                            gt_answer = normalize_answer(gt_answer_raw)
                            
                            # ChatGPT答案
                            chatgpt_answer = chatgpt_answers.get(qid, 'No')
                            
                            chatgpt_app_data['answers'][qid] = {
                                'gt_answer': gt_answer,
                                'chatgpt_answer': chatgpt_answer
                            }
                        
                        chatgpt_apps_data.append(chatgpt_app_data)
                        print(f"   ✅ ChatGPT处理完成")
                    
                    except Exception as e:
                        print(f"   ❌ 处理ChatGPT文件 {chatgpt_filename} 时出错: {e}")
                        continue
                
                # 创建ChatGPT对比表格（按问题分组）
                print(f"\n📋 创建ChatGPT对比表格（按问题分组）...")
                
                for qid in questions_order:
                    question_label = question_labels[qid]
                    print(f"   添加所有应用的 {question_label} 问题...")
                    
                    for app_data in chatgpt_apps_data:
                        if qid in app_data['answers']:
                            answer_data = app_data['answers'][qid]
                            
                            chatgpt_comparisons.append({
                                'Question': question_label,
                                'App': app_data['app_id'],
                                'Ground Truth': answer_data['gt_answer'],
                                'ChatGPT': answer_data['chatgpt_answer']
                            })
                
            else:
                print(f"⚠️ ChatGPT输出文件夹存在但为空")
        except Exception as e:
            print(f"❌ 读取ChatGPT文件夹时出错: {e}")
    else:
        print(f"⚠️ ChatGPT数据文件夹不存在: {chatgpt_outputs}")
        print(f"   将在您上传ChatGPT数据后生成ChatGPT对比表格")
    
    # 如果ChatGPT数据不可用，创建占位符表格
    if not chatgpt_available:
        print(f"\n📋 创建ChatGPT占位符表格...")
        for qid in questions_order:
            question_label = question_labels[qid]
            
            for app_data in all_apps_data:
                if qid in app_data['answers']:
                    answer_data = app_data['answers'][qid]
                    
                    chatgpt_comparisons.append({
                        'Question': question_label,
                        'App': app_data['app_id'],
                        'Ground Truth': answer_data['gt_answer'],
                        'ChatGPT': 'To be imported'
                    })
    
    # 7. 创建DataFrame
    rag_df = pd.DataFrame(rag_comparisons) if rag_comparisons else None
    chatgpt_df = pd.DataFrame(chatgpt_comparisons) if chatgpt_comparisons else None
    
    # 8. 保存表格
    if rag_df is not None:
        try:
            rag_output_file = os.path.join(output_dir, 'rag_vs_groundtruth_table.csv')
            rag_df.to_csv(rag_output_file, index=False, encoding='utf-8')
            print(f"\n💾 RAG系统对比表格已保存到: rag_vs_groundtruth_table.csv")
        except PermissionError:
            print(f"\n⚠️ 保存RAG表格时权限被拒绝，请关闭可能正在使用该文件的程序")
            print(f"   文件路径: {rag_output_file}")
        except Exception as e:
            print(f"\n❌ 保存RAG表格时出错: {e}")
    
    if chatgpt_df is not None:
        try:
            chatgpt_output_file = os.path.join(output_dir, 'chatgpt_vs_groundtruth_table.csv')
            chatgpt_df.to_csv(chatgpt_output_file, index=False, encoding='utf-8')
            if chatgpt_available:
                print(f"💾 ChatGPT对比表格已保存到: chatgpt_vs_groundtruth_table.csv")
            else:
                print(f"💾 ChatGPT占位符表格已保存到: chatgpt_vs_groundtruth_table.csv")
                print(f"   ⚠️ ChatGPT列显示'To be imported'，请上传ChatGPT数据后重新运行")
        except PermissionError:
            print(f"\n⚠️ 保存ChatGPT表格时权限被拒绝，请关闭可能正在使用该文件的程序")
            print(f"   文件路径: {chatgpt_output_file}")
        except Exception as e:
            print(f"\n❌ 保存ChatGPT表格时出错: {e}")
    
    # 9. 统计信息
    if rag_df is not None:
        total_rag = len(rag_df)
        print(f"\n📊 RAG系统统计:")
        print(f"   - 总记录数: {total_rag}")
        print(f"   - 处理应用数: {len(all_apps_data)}")
        print(f"   - 问题类型: Q1, Q3, Q4, Q5")
        print(f"   - 表格格式: 按问题分组（先所有Q1，再所有Q3，以此类推）")
        
    print(f"\n📁 ChatGPT数据路径: {chatgpt_outputs}")
    if not chatgpt_available:
        print(f"   💡 请将ChatGPT数据上传到上述路径，然后重新运行代码")
    
    return rag_df, chatgpt_df

# 执行表格生成
rag_table, chatgpt_table = create_comparison_tables()

🚀 开始生成对比表格...
✅ 成功加载 Ground Truth: 213 条记录
✅ 找到 205 个RAG数据文件
📁 文件列表: ['1019230398.json', '1063248233.json', '1112006222.json', '1155742410.json', '1157882317.json', '1158555867.json', '1193006281.json', '1212999482.json', '1218240628.json', '1234966740.json', '1236566346.json', '1273620738.json', '1276818064.json', '1289018369.json', '1291472331.json', '1300107718.json', '1312926037.json', '1321563535.json', '1332580637.json', '1361356590.json', '1373639965.json', '1387949872.json', '1396647655.json', '1412024302.json', '1421810571.json', '1435692352.json', '1436471710.json', '1441112707.json', '1444463273.json', '1453180679.json', '1458156758.json', '1458846512.json', '1460599496.json', '1463607627.json', '1465984722.json', '1478859313.json', '1485424497.json', '1489944782.json', '1490281801.json', '1490544446.json', '1491792162.json', '1492803107.json', '1493155192.json', '1494699802.json', '1498229813.json', '1498350923.json', '1499411785.json', '1500085577.json', '1501731777.json',

In [2]:
# 📊 显示生成的表格
def display_tables():
    """
    显示生成的两个对比表格
    """
    print("="*80)
    print("📋 生成的对比表格预览")
    print("="*80)
    
    # 显示RAG系统对比表格
    if rag_table is not None and len(rag_table) > 0:
        print("\n🤖 表格1: RAG系统 vs Ground Truth")
        print("-" * 60)
        print("📌 格式说明: 按问题分组显示（Q1→Q3→Q4→Q5）")
        print(rag_table.to_string(index=False))
        
        # 计算RAG系统准确率
        rag_correct = sum(1 for _, row in rag_table.iterrows() 
                         if row['Ground Truth'] == row['Our RAG System'])
        rag_total = len(rag_table)
        rag_accuracy = rag_correct / rag_total * 100 if rag_total > 0 else 0
        
        print(f"\n📊 RAG系统统计:")
        print(f"   - 正确匹配: {rag_correct}/{rag_total}")
        print(f"   - 准确率: {rag_accuracy:.1f}%")
    else:
        print("\n❌ RAG系统表格生成失败")
    
    # 显示ChatGPT对比表格
    if chatgpt_table is not None and len(chatgpt_table) > 0:
        print(f"\n💬 表格2: ChatGPT vs Ground Truth")
        print("-" * 60)
        print("📌 格式说明: 按问题分组显示（Q1→Q3→Q4→Q5）")
        
        # 检查是否有实际的ChatGPT数据
        has_real_data = not all(row['ChatGPT'] == 'To be imported' for _, row in chatgpt_table.iterrows())
        
        if has_real_data:
            print("✅ 使用实际ChatGPT数据")
        else:
            print("⚠️ ChatGPT数据尚未上传，显示占位符")
        
        print(chatgpt_table.to_string(index=False))
        
        # 只有在有实际数据时才计算准确率
        if has_real_data:
            chatgpt_correct = sum(1 for _, row in chatgpt_table.iterrows() 
                                 if row['Ground Truth'] == row['ChatGPT'])
            chatgpt_total = len(chatgpt_table)
            chatgpt_accuracy = chatgpt_correct / chatgpt_total * 100 if chatgpt_total > 0 else 0
            
            print(f"\n📊 ChatGPT统计:")
            print(f"   - 正确匹配: {chatgpt_correct}/{chatgpt_total}")
            print(f"   - 准确率: {chatgpt_accuracy:.1f}%")
        else:
            print(f"\n📊 ChatGPT统计:")
            print(f"   - 数据状态: 等待上传")
            print(f"   - 占位符记录数: {len(chatgpt_table)}")
    else:
        print("\n❌ ChatGPT表格生成失败")
    
    # 显示保存文件信息
    output_dir = project_root / "final outputs"
    print(f"\n📁 文件保存位置: {output_dir}")
    print("   - rag_vs_groundtruth_table.csv")
    print("   - chatgpt_vs_groundtruth_table.csv")

# 显示表格
display_tables()

📋 生成的对比表格预览

🤖 表格1: RAG系统 vs Ground Truth
------------------------------------------------------------
📌 格式说明: 按问题分组显示（Q1→Q3→Q4→Q5）
Question        App Ground Truth Our RAG System
      Q1 1019230398          Yes            Yes
      Q1 1063248233          Yes            Yes
      Q1 1112006222          Yes            Yes
      Q1 1155742410          Yes             No
      Q1 1157882317          Yes            Yes
      Q1 1158555867          Yes            Yes
      Q1 1193006281          Yes            Yes
      Q1 1212999482          Yes            Yes
      Q1 1218240628          Yes            Yes
      Q1 1234966740          Yes            Yes
      Q1 1236566346          Yes            Yes
      Q1 1273620738          Yes            Yes
      Q1 1276818064          Yes            Yes
      Q1 1289018369          Yes            Yes
      Q1 1291472331          Yes            Yes
      Q1 1300107718          Yes            Yes
      Q1 1312926037          Yes            Yes
    

In [3]:
# 📈 按问题类型详细分析
def analyze_by_questions():
    """
    按问题类型进行详细分析
    """
    print("\n" + "="*80)
    print("📈 按问题类型详细分析")
    print("="*80)
    
    if rag_table is not None and len(rag_table) > 0:
        print("\n🔍 RAG系统按问题分析:")
        
        # 按问题分组分析
        for question in ['Q1', 'Q3', 'Q4', 'Q5']:
            q_data = rag_table[rag_table['Question'] == question]
            if len(q_data) > 0:
                q_correct = sum(1 for _, row in q_data.iterrows() 
                               if row['Ground Truth'] == row['Our RAG System'])
                q_total = len(q_data)
                q_accuracy = q_correct / q_total * 100 if q_total > 0 else 0
                
                print(f"\n   📋 {question} 问题:")
                print(f"      - 应用数量: {q_total}")
                print(f"      - 正确匹配: {q_correct}")
                print(f"      - 准确率: {q_accuracy:.1f}%")
                
                # 显示错误案例
                q_errors = q_data[q_data['Ground Truth'] != q_data['Our RAG System']]
                if len(q_errors) > 0:
                    print(f"      ❌ 错误案例:")
                    for _, row in q_errors.iterrows():
                        print(f"         • {row['App']}: GT='{row['Ground Truth']}' vs RAG='{row['Our RAG System']}'")
                else:
                    print(f"      ✅ 无错误案例")
    
    # 验证表格格式是否正确
    print(f"\n✅ 表格格式验证:")
    if rag_table is not None:
        question_counts = rag_table['Question'].value_counts()
        print(f"   - 各问题的记录数: {dict(question_counts)}")
        
        # 检查是否按问题分组
        questions_order = []
        for _, row in rag_table.iterrows():
            if len(questions_order) == 0 or questions_order[-1] != row['Question']:
                questions_order.append(row['Question'])
        
        print(f"   - 问题显示顺序: {' → '.join(questions_order)}")
        
        if questions_order == ['Q1', 'Q3', 'Q4', 'Q5']:
            print(f"   ✅ 表格格式正确：按问题分组显示")
        else:
            print(f"   ⚠️ 表格格式可能不正确")

# 运行分析
analyze_by_questions()


📈 按问题类型详细分析

🔍 RAG系统按问题分析:

   📋 Q1 问题:
      - 应用数量: 205
      - 正确匹配: 197
      - 准确率: 96.1%
      ❌ 错误案例:
         • 1155742410: GT='Yes' vs RAG='No'
         • 1511247522: GT='Yes' vs RAG='No'
         • 1564740564: GT='Yes' vs RAG='No'
         • 6472111765: GT='Yes' vs RAG='No'
         • 6473814624: GT='Yes' vs RAG='N/A'
         • 6473851170: GT='No' vs RAG='Yes'
         • 6476922373: GT='Yes' vs RAG='No'
         • 659065074: GT='Yes' vs RAG='No'

   📋 Q3 问题:
      - 应用数量: 205
      - 正确匹配: 200
      - 准确率: 97.6%
      ❌ 错误案例:
         • 1212999482: GT='Yes' vs RAG='No'
         • 1658145102: GT='Yes' vs RAG='No'
         • 6456222443: GT='Yes' vs RAG='No'
         • 6473851170: GT='No' vs RAG='Yes'
         • 659065074: GT='Yes' vs RAG='No'

   📋 Q4 问题:
      - 应用数量: 205
      - 正确匹配: 180
      - 准确率: 87.8%
      ❌ 错误案例:
         • 1157882317: GT='No' vs RAG='Yes'
         • 1276818064: GT='No' vs RAG='Yes'
         • 1435692352: GT='No' vs RAG='Yes'
         • 1494699802: 